In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from env import robot_goal
import random
from tqdm import tqdm
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt


In [11]:
env = robot_goal()

In [12]:
"""env.reset()

done = False
total_reward = 0
simulation_start = 0
while simulation_start != 1:
    simulation_start = env.sim.startSimulation()
while not done:
   action = random.randint(0,2)
   obs, rew, done = env.step(action)
   total_reward += rew
   print(f"{obs} -> {rew}")
simulation_end = 0        
while simulation_end != 1:
    simulation_end = env.sim.stopSimulation()     
print(f"Total reward: {total_reward}")"""



'env.reset()\n\ndone = False\ntotal_reward = 0\nsimulation_start = 0\nwhile simulation_start != 1:\n    simulation_start = env.sim.startSimulation()\nwhile not done:\n   action = random.randint(0,2)\n   obs, rew, done = env.step(action)\n   total_reward += rew\n   print(f"{obs} -> {rew}")\nsimulation_end = 0        \nwhile simulation_end != 1:\n    simulation_end = env.sim.stopSimulation()     \nprint(f"Total reward: {total_reward}")'

In [13]:
num_inputs = 10
num_actions = 3

num_hidden = 128

inputs = keras.layers.Input(shape=(num_inputs,))
common = keras.layers.Dense(num_hidden, activation="relu")(inputs)
action = keras.layers.Dense(num_actions, activation="softmax")(common)
critic = keras.layers.Dense(1)(common)

model = keras.Model(inputs=inputs, outputs=[action, critic])

In [14]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 10)]                 0         []                            
                                                                                                  
 dense_3 (Dense)             (None, 128)                  1408      ['input_2[0][0]']             
                                                                                                  
 dense_4 (Dense)             (None, 3)                    387       ['dense_3[0][0]']             
                                                                                                  
 dense_5 (Dense)             (None, 1)                    129       ['dense_3[0][0]']             
                                                                                            

In [15]:

def run_episode(max_steps_per_episode=10000, render=False):
    states, actions, probs, rewards, critic = [], [], [], [], []  # Inicialização de listas para armazenar informações durante o episódio
    state = env.reset()  # Reinicialização do ambiente e obtenção do estado inicial
    simulation_start = 0
    while simulation_start != 1:
        simulation_start = env.sim.startSimulation()  # Inicialização da simulação
        
    for _ in tqdm(range(max_steps_per_episode)):  # Loop para cada etapa do episódio
        state = np.reshape(state, (1, state.shape[0]))  # Reformatação do estado para atender aos requisitos do modelo
        action_probs, est_rew = model(state)  # Obtenção das probabilidades de ação e da estimativa de recompensa do modelo
        action = np.argmax(action_probs)  # Escolha da ação com a maior probabilidade
        nstate, reward, done = env.step(action)  # Execução da ação no ambiente e obtenção do próximo estado, recompensa e indicador de conclusão
        
        if done:
            break  # Se o episódio estiver concluído, encerra o loop
        
        # Armazenamento das informações do episódio
        states.append(state)
        actions.append(action)
        probs.append(tf.math.log(action_probs[0, action]))
        rewards.append(reward)
        critic.append(est_rew[0, 0])
        
        state = nstate  # Atualização do estado para o próximo estado
        
    return states, actions, probs, rewards, critic


In [16]:
optimizer = keras.optimizers.Adam(learning_rate=0.01)
huber_loss = keras.losses.Huber()
episode_count = 0
running_reward = 0

while True:
    state = env.reset()
    episode_reward = 0
    with tf.GradientTape() as tape:
        _,_,action_probs, rewards, critic_values = run_episode()
        episode_reward = np.sum(rewards)

        # Atualizar recompensa em execução para verificar condição de resolução.
        running_reward = 0.05 * episode_reward + (1 - 0.05) * running_reward

        # Calculando valores de perda para atualizar nossa rede.
        actor_losses = []
        critic_losses = []
        for log_prob, value, rew in zip(action_probs, critic_values, rewards):
            # Quando tomamos a ação com probabilidade log_prob, recebemos uma
            # recompensa descontada de rew, enquanto o crítico a previu como value. 
            # Primeiro, calculamos a perda do ator, para fazer com que o ator preveja ações que levem a recompensas mais altas.
            diff = rew - value
            actor_losses.append(-log_prob * diff)

            # A perda do crítico visa minimizar a diferença entre a recompensa prevista 'value' e a recompensa descontada real 'rew'.
            critic_losses.append(
                huber_loss(tf.expand_dims(value, 0), tf.expand_dims(rew, 0))
            )

        # Backpropagation
        loss_value = sum(actor_losses) + sum(critic_losses)
        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Logs
    episode_count += 1
    if episode_count % 10 == 0:
        model.save("model.h5")
        template = "running reward: {:.2f} at episode {}"
        print(template.format(running_reward, episode_count))

    if running_reward > 195:  # Condition to consider the task solved
        print("Solved at episode {}!".format(episode_count))
        break


 41%|████      | 4077/10000 [41:35<1:18:59,  1.25it/s]